# Monitoring

Azure AI Agent Service supports tracing where all requests can be sent to Application Insights and can be used to monitor agents.

This notebook shows how to do it, but Application Insights was not provisioned automatically. If you want to follow this notebook you need to deploy it by yourself. You can find more information on how to do it [here](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/tracing).

This tracing capability requires some Python libraries which the code below will install.

In [ ]:
%pip install opentelemetry
%pip install azure-core-tracing-opentelemetry
%pip install azure-monitor-opentelemetry

Import some Python classes, functions and definitions.

In [ ]:
import os, time, json
from pprint import pp as pp

from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.ai.projects.models import FunctionTool, RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput
from typing import Any, Callable, Set

# Load environment variables from .env file
load_dotenv(override=True)

def pprint(obj):
    pp(obj.as_dict() if hasattr(obj, "as_dict") else obj, width=100)

# Print the environments we will be using.
print(f"PROJECT_CONNECTION_STRING: {os.getenv('PROJECT_CONNECTION_STRING')}")
print(f"BING_CONNECTION_NAME: {os.getenv('BING_CONNECTION_NAME')}")  

Get the project reference using the current authenticated user and the connection string to the project where the agents will be created. 

In [3]:
project = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

Get the Application Insights connection string linked to the Azure AI Foundry project and configure Azure Monitor.

In [4]:
application_insights_connection_string = project.telemetry.get_connection_string()

if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()
    
configure_azure_monitor(connection_string=application_insights_connection_string)

The code below will define a function that will be used by our agent to get weather information.

This code was instrumented using the **trace** class from OpenTelemetry to generate information about its execution.

In [5]:
scenario = os.path.basename("Weather Agent")
tracer = trace.get_tracer(__name__)

# The tracer.start_as_current_span decorator will trace the function call and enable adding additional attributes
# to the span in the function implementation. Note that this will trace the function parameters and their values.
@tracer.start_as_current_span("fetch_weather")  # type: ignore
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C",
                         "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}

    # Adding attributes to the current span
    span = trace.get_current_span()
    span.set_attribute("requested_location", location)

    weather = mock_weather_data.get(
        location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json


# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather,
}

Now, the last piece of the puzzle, let's create an agent, and ask him about the weather in New York.  During its execution telemetry will be collected and pushed to Application Insights.

In [ ]:
# Initialize function tool with user function
functions = FunctionTool(functions=user_functions)

with tracer.start_as_current_span(scenario):    
    
    agent = project.agents.create_agent(
        model="gpt-4o-mini",
        name="Weather Agent",
        instructions="You are a helpful agent which help people find information about the weather.",
        tools=functions.definitions,
    )    

    thread = project.agents.create_thread()

    message = project.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hello, what is the weather in New York?",
    )

    run = project.agents.create_run(
        thread_id=thread.id,
        assistant_id=agent.id
    )
    
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project.agents.get_run(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project.agents.cancel_run(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project.agents.submit_tool_outputs_to_run(
                    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                )

            print(f"Current run status: {run.status}")    

After a few seconds go to Azure AI Foundry, select the project where the telemetry was pushed and in the tracing option (highlighted in yellow) you will find the telemetry that was collected and sent.

<img src="./assets/img/ai-foundry-tracing.png" width="70%">

You have reached the end of this lab. 👏